In [133]:
import numpy as np
import tensorflow as tf

from keras.models import Sequential

from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import Lambda

from keras.layers import GlobalAveragePooling2D #PSROIPooling使用

import keras.backend as K

In [134]:
k=3
C=20 #for VOC

In [135]:
#[x1 y1 x2 y2]
roi = np.random.randint(low=0 , high=599 , size=(300 , 4)) #模拟rois

In [136]:
roi = roi[roi[: , 0] < roi[: , 2]] #x不满足的 过滤掉
roi = roi[roi[: , 1] < roi[: , 3]] #y不满足的 过滤掉

In [137]:
# idx = np.arange(0 , roi.shape[0])
idx = np.zeros(shape=roi.shape[0]) #只作用与一张图片 故索引为0 参考tf.image.crop_and_resize函数
idx = np.expand_dims(idx , axis=-1)

In [138]:
roi = np.concatenate((idx , roi) , axis=-1) #制造出来合格的有RPN产生的ROIs

In [139]:
strides = 600 / 20 #累计stride

#roi需要根据累计的stride 向feature map映射
roi[: , 1:] = roi[: , 1:] // strides #完成映射 feature map

roi = roi.astype('int')

In [140]:
model = Sequential()

#resnet的100层输出
resnet_output = Input(shape=(20,20,3)) #假定已经为resnet的输出

reduce_d = Conv2D(filters=1024 , kernel_size=(1,1) , strides=(1,1) , activation='relu')(resnet_output) #降维增加的

#new
#for entire image
cls = Conv2D(filters=k*k*(C+1) , kernel_size=(1,1) , strides=(1,1) , activation='relu')(reduce_d)
loc = Conv2D(filters=k*k*4 , kernel_size=(1,1) , strides=(1,1) , activation=None)(reduce_d)

In [141]:
#feature_map_h = cls.get_shape().as_list()[1]
#feature_map_w = cls.get_shape().as_list()[2]

In [142]:
#reference https://blog.csdn.net/m0_38024332/article/details/81779544
#tf.image.crop_resize中boxes信息为相对图片的比例信息 且顺序为 [ y1,x1,y2,x2 ]

## step 1:
#roi[1] = roi[1] / feature_map_w
#roi[3] = roi[3] / feature_map_w
#
## step 2:
#roi[2] = roi[2] / feature_map_h
#roi[4] = roi[4] / feature_map_h
#
#roi[: , [1,2,3,4]] = roi[: , [2,1,4,3]]

In [143]:
def psroipooling(k_2_grids):
    k_2_grids = GlobalAveragePooling2D()(k_2_grids)
    #全局平均池化后维度发生变化 增加两个维度 height and width
    k_2_grids = tf.expand_dims(k_2_grids , axis=0)
    k_2_grids = tf.expand_dims(k_2_grids , axis=0)    
    
    return k_2_grids

In [144]:
def score_map_and_psroipooling(feature_map , dim):
    '''
    feature_map: 两种可能 分类的 定位的
    dim: 分类时为21 定位时为4
    '''

    #所有roi
    psrps = []

    for one_roi in roi:
        #一个roi
        one_roi_w = one_roi[3] - one_roi[1]
        one_roi_h = one_roi[4] - one_roi[2]

        t_l = tf.slice(feature_map , begin=[0 , one_roi[2] , one_roi[1] , 21*0] , size=[1 , one_roi_h//3 , one_roi_w//3 , dim])
        t_c = tf.slice(feature_map , begin=[0 , one_roi[2] , one_roi[1]+one_roi_w//3,21*1] , size=[1 , one_roi_h//3 , one_roi_w//3 , dim])
        t_r = tf.slice(feature_map , begin=[0 , one_roi[2] , one_roi[1]+one_roi_w//3*2,21*2] , size=[1 , one_roi_h//3 , one_roi_w//3 , dim])
        c_l = tf.slice(feature_map , begin=[0 , one_roi[2] + one_roi_h//3*2 , one_roi[1] ,  21*3] , size=[1 , one_roi_h//3 , one_roi_w//3 , dim])
        c_c = tf.slice(feature_map , begin=[0 , one_roi[2] + one_roi_h//3*2 , one_roi[1]+one_roi_w//3 ,  21*4] , size=[1 , one_roi_h//3 , one_roi_w//3 , dim])
        c_r = tf.slice(feature_map , begin=[0 , one_roi[2] + one_roi_h//3*2 , one_roi[1]+one_roi_w//3*2 ,  21*5] , size=[1 , one_roi_h//3 , one_roi_w//3 , dim])
        d_l = tf.slice(feature_map , begin=[0 , one_roi[2] + one_roi_h//3 , one_roi[1] ,  21*6] , size=[1 , one_roi_h//3 , one_roi_w//3 , dim])
        d_c = tf.slice(feature_map , begin=[0 , one_roi[2] + one_roi_h//3 , one_roi[1]+one_roi_w//3 ,  21*7] , size=[1 , one_roi_h//3 , one_roi_w//3 , dim])
        d_r = tf.slice(feature_map , begin=[0 , one_roi[2] + one_roi_h//3 , one_roi[1]+one_roi_w//3*2 ,  21*8] , size=[1 , one_roi_h//3 , one_roi_w//3 , dim])

        #PSROIPooling
        t_l = psroipooling(t_l)
        t_c = psroipooling(t_c)
        t_r = psroipooling(t_r)
        c_l = psroipooling(c_l)
        c_c = psroipooling(c_c)
        c_r = psroipooling(c_r)
        d_l = psroipooling(d_l)
        d_c = psroipooling(d_c)
        d_r = psroipooling(d_r)

        psrp = tf.concat([t_l,t_c,t_r,c_l,c_c,c_r,d_l,d_c,d_r] , axis=0)
        psrp = tf.reshape( psrp , shape=[1,3,3,dim])

        psrps.append(psrp)
    
    psrps = tf.concat(psrps , axis=0)    
    voting = GlobalAveragePooling2D()(psrps)
    
    if dim == (C+1):
        #分类概率信息
        return tf.nn.softmax(voting)
    else:
        #定位offset信息
        return voting

In [145]:
probs = score_map_and_psroipooling(cls , dim=21)

In [146]:
offset = score_map_and_psroipooling(loc , dim=4)

In [147]:
offset.get_shape()

TensorShape([Dimension(71), Dimension(4)])

In [148]:
probs.get_shape()

TensorShape([Dimension(71), Dimension(21)])